# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#----------------------------------------------------------------------------------------------------
# Note:  It is actually easier to determine many of these values based on the source tables of
# school_data & student_data... but I decided to walk the path of most resistance and work with the
# complete table created above
#----------------------------------------------------------------------------------------------------

# Calculate the total number of schools
numSchools = len(school_data_complete["school_name"].unique())

# Calculate the total number of students
numStudents = len(school_data_complete)

# Calculate the total budget
totalBudget = (school_data_complete["budget"].unique()).sum()
# Note that this assumes that all schools have different budgets, this wouldn't work if two of the schools have exactly
#     the same budget number.  I've tested unique() and gotten 15 different budgets for 15 different schools

# Calculate the average math score
aveMathScore = school_data_complete["math_score"].mean()

# Calculate the average read score
aveReadingScore = school_data_complete["reading_score"].mean()

# Calculate the overall passing rate (note passing rate is 70% or more)
# Note that I am now adding "overall_score" to the complete table for future use
school_data_complete["overall_score"] = (school_data_complete["math_score"] + school_data_complete["reading_score"]) / 2
pctPassOverall = len(school_data_complete[school_data_complete["overall_score"] >= 70]) / numStudents * 100
# I need to explain why I did it this way.  Statistically, adding the average math score with the average reading score
#     for the district isn't a strong indicator of anything.  As well, it doesn't really talk about "Passing" considering
#     the next values for % Passing Math & % Passing Reading applies a standard of 70 to pass.  Adding averages for the
#     overall district don't indicate how many students would have actually passed inidvidually.
#     So instead, I interpreted % Overall Passing Rate to be what percentage of students would have 70 or more
#     on the OVERALL score which is the average between the individual math and reading scores.

# Calculate the percentage of students with a passing math score (70 or greater)
pctPassMath = len(school_data_complete[school_data_complete["math_score"] >= 70]) / numStudents * 100

# Calculate the percentage of students with a passing reading score (70 or greater)
pctPassReading = len(school_data_complete[school_data_complete["reading_score"] >= 70]) / numStudents * 100

# Create a dataframe to hold the above results
dsummary_df = pd.DataFrame([{"Total Schools" : numSchools,
                             "Total Students" : numStudents,
                             "Total Budget" : totalBudget,
                             "Average Math Score" : aveMathScore,
                             "Average Reading Score" : aveReadingScore,
                             "% Passing Math" : pctPassMath,
                             "% Passing Reading" : pctPassReading,
                             "% Overall Passing Rate" : pctPassOverall
                             }])

# Optional: give displayed data cleaner formatting
dsummary_df["Total Schools"] = dsummary_df["Total Schools"].map('{:,}'.format)
dsummary_df["Total Students"] = dsummary_df["Total Students"].map('{:,}'.format)
dsummary_df["Total Budget"] = dsummary_df["Total Budget"].map('${:,.2f}'.format)
dsummary_df["Average Math Score"] = dsummary_df["Average Math Score"].map('{:,.1f}'.format)
dsummary_df["Average Reading Score"] = dsummary_df["Average Reading Score"].map('{:,.1f}'.format)
dsummary_df["% Passing Math"] = dsummary_df["% Passing Math"].map('{:,.1f}%'.format)
dsummary_df["% Passing Reading"] = dsummary_df["% Passing Reading"].map('{:,.1f}%'.format)
dsummary_df["% Overall Passing Rate"] = dsummary_df["% Overall Passing Rate"].map('{:,.1f}%'.format)

# reorder columns to match what is seen in original starter file
district_summary = dsummary_df[["Total Schools", "Total Students", "Total Budget", "Average Math Score", "Average Reading Score",
                               "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]

district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",79.0,81.9,75.0%,85.8%,89.4%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [3]:
# Create school summary dataframe from school data and include School Name, School Type, Total Students, & Total School Budget
school_summary_df = school_data[["school_name", "type", "size", "budget"]]

# Add Per Student Budget column
school_summary_df["Per Student Budget"] = school_summary_df["budget"] / school_summary_df["size"]

grp_student_data = student_data.groupby(["school_name"], as_index = False)

# Create dataframe for Average Math Score per school
aveMath_PerSchool = pd.DataFrame(grp_student_data["math_score"].mean())

# Create dataframe for Average Reading Score per school
aveReading_PerSchool = pd.DataFrame(grp_student_data["reading_score"].mean())

# To determine % Passing Math, create dataFrames showing the number of students who passed Math per school
passMath = student_data[student_data['math_score'] >= 70]
grpPassMath = passMath.groupby(["school_name"], as_index = False)
schoolPassMath = pd.DataFrame(grpPassMath["math_score"].count())
# Note this will be merged with school summary to eventually calculate % Passing Math

# To determine % Passing Reading, create dataFrames showing the number of students who passed Reading per school
passReading = student_data[student_data['reading_score'] >= 70]
grpPassReading = passReading.groupby(["school_name"], as_index = False)
schoolPassReading = pd.DataFrame(grpPassReading["reading_score"].count())
# Note this will be merged with school summary to eventually calculate % Passing Reading

# To determine % Overall Passing Rate per school, create data frame showing students with a passing overall score
overall_df = student_data.copy()
overall_df['overall_score'] = ( overall_df['math_score'] + overall_df['reading_score'] ) / 2
passOverall = overall_df[overall_df['overall_score'] >= 70]
grpPassOverall = passOverall.groupby(["school_name"], as_index = False)
schoolPassOverall = pd.DataFrame(grpPassOverall["overall_score"].count())
# Note this will be merged with school summary to eventually calculate % Passing Reading

# Merge all the dataframes
#df1 = pd.merge(school_summary_df, aveMath_PerSchool, how="left", on=["school_name", "school_name"])
df1 = pd.merge(school_summary_df, aveMath_PerSchool, how="outer", on=["school_name"])
df2 = pd.merge(df1, aveReading_PerSchool, how="outer", on=["school_name"])
df3 = pd.merge(df2, schoolPassMath, how="outer", on=["school_name"])
df4 = pd.merge(df3, schoolPassReading, how="outer", on=["school_name"])
df5 = pd.merge(df4, schoolPassOverall, how="outer", on=["school_name"])

# Calculate % Passing Math per school
df5['% Passing Math'] = df5['math_score_y'] / df5['size'] * 100

# Calculate % Passing Reading per school
df5['% Passing Reading'] = df5['reading_score_y'] / df5['size'] * 100

# Calculate % Overall Passing Rate per school
df5['% Overall Passing Rate'] = df5['overall_score'] / df5['size'] * 100

# Rename column headers to match example
df5 = df5.rename(columns = {'school_name' : 'School Name',
                            'type' : 'School Type',
                            'size' : 'Total Students',
                            'budget' : 'Total School Budget',
                            'math_score_x' : 'Average Math Score',
                            'reading_score_x' : 'Average Reading Score'
                           })

# create copy to format columns for cleaner display
# I did this to avoid this warning I kept getting:  SettingWithCopyWarning: 
#    A value is trying to be set on a copy of a slice from a DataFrame.
#    Try using .loc[row_indexer,col_indexer] = value instead
df6 = df5.copy()

# Optional Format columns for cleaner display
df6["Total Students"] = df6["Total Students"].map('{:,}'.format)
df6["Total School Budget"] = df6["Total School Budget"].map('${:,.2f}'.format)
df6["Per Student Budget"] = df6["Per Student Budget"].map('${:,.2f}'.format)
df6["Average Math Score"] = df6["Average Math Score"].map('{:,.1f}'.format)
df6["Average Reading Score"] = df6["Average Reading Score"].map('{:,.1f}'.format)
df6["% Passing Math"] = df6["% Passing Math"].map('{:,.1f}%'.format)
df6["% Passing Reading"] = df6["% Passing Reading"].map('{:,.1f}%'.format)
df6["% Overall Passing Rate"] = df6["% Overall Passing Rate"].map('{:,.1f}%'.format)

# Copy needed columns in the correct order to final school summary dataframe
school_summary = df6[["School Name", "School Type", "Total Students", "Total School Budget", "Per Student Budget",
                      "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading",
                      "% Overall Passing Rate"]]

school_summary.sort_values("School Name")
school_summary.set_index("School Name", inplace=True)

# Get Top 5 schools
desc_school_summary = school_summary.sort_values('% Overall Passing Rate', ascending = False)
desc_school_summary.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.1,84.0,94.1%,97.0%,99.6%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.4,83.8,93.4%,97.1%,99.5%
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.4,83.7,93.9%,95.9%,99.4%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.3,84.0,93.9%,96.5%,99.3%
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,94.6%,95.9%,99.2%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [4]:
asc_school_summary = school_summary.sort_values('% Overall Passing Rate', ascending = True)
asc_school_summary.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7,81.2,66.0%,80.7%,84.7%
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.8,80.7,66.4%,80.2%,84.7%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.1,80.7,68.3%,79.3%,84.8%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.3,80.9,66.8%,80.9%,84.9%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6,81.2,65.7%,81.3%,85.0%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [5]:
# Setup a dataframe for each grade
grade09_df = student_data[student_data["grade"] == '9th'].groupby(["school_name"])
grade10_df = student_data[student_data["grade"] == '10th'].groupby(["school_name"])
grade11_df = student_data[student_data["grade"] == '11th'].groupby(["school_name"])
grade12_df = student_data[student_data["grade"] == '12th'].groupby(["school_name"])

# Get the series for the mean math score for each school
grp_math09 = grade09_df["math_score"].mean()
grp_math10 = grade10_df["math_score"].mean()
grp_math11 = grade11_df["math_score"].mean()
grp_math12 = grade12_df["math_score"].mean()

# Create dataframe
aveMathScore_PerGrade = pd.DataFrame({"9th" : grp_math09,
                                      "10th" : grp_math10,
                                      "11th" : grp_math11,
                                      "12th" : grp_math12
                                     })

# Apply formatting
aveMathScore_PerGrade["9th"] = aveMathScore_PerGrade["9th"].map('{:,.2f}'.format)
aveMathScore_PerGrade["10th"] = aveMathScore_PerGrade["10th"].map('{:,.2f}'.format)
aveMathScore_PerGrade["11th"] = aveMathScore_PerGrade["11th"].map('{:,.2f}'.format)
aveMathScore_PerGrade["12th"] = aveMathScore_PerGrade["12th"].map('{:,.2f}'.format)
del aveMathScore_PerGrade.index.name

# Display dataframe
aveMathScore_PerGrade


,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [6]:
# Get the series for the mean reading score for each school
grp_read09 = grade09_df["reading_score"].mean()
grp_read10 = grade10_df["reading_score"].mean()
grp_read11 = grade11_df["reading_score"].mean()
grp_read12 = grade12_df["reading_score"].mean()

# Create dataframe
aveReadScore_PerGrade = pd.DataFrame({"9th" : grp_read09,
                                      "10th" : grp_read10,
                                      "11th" : grp_read11,
                                      "12th" : grp_read12
                                     })

# Apply formatting
aveReadScore_PerGrade["9th"] = aveReadScore_PerGrade["9th"].map('{:,.2f}'.format)
aveReadScore_PerGrade["10th"] = aveReadScore_PerGrade["10th"].map('{:,.2f}'.format)
aveReadScore_PerGrade["11th"] = aveReadScore_PerGrade["11th"].map('{:,.2f}'.format)
aveReadScore_PerGrade["12th"] = aveReadScore_PerGrade["12th"].map('{:,.2f}'.format)
del aveReadScore_PerGrade.index.name

# display dataframe
aveReadScore_PerGrade

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [18]:
# remove any unnecessary columns from school summary pdf prior to formatting (pd5)

"""
del df5["math_score_y"]
del df5["reading_score_y"]
del df5["overall_score"]
"""

# create new table that has student scores in relation with school budget per student
student_school_df = pd.merge(school_data_complete, df5, how='left', left_on ='school_name', right_on ='School Name')

# remove unneeded columns in student_school_df
del student_school_df['school_name']
del student_school_df['type']
del student_school_df['size']
del student_school_df['budget']
del student_school_df['Average Math Score']
del student_school_df['Average Reading Score']
del student_school_df['% Passing Math']
del student_school_df['% Passing Reading']
del student_school_df['% Overall Passing Rate']

# Create Bins & Labels
spending_bins = [0, 580, 600, 620, 640, 660, 680]
group_names = ['\$0 - \$580', '\$580 - \$600', '\$600 - \$620', '\$620 - \$640', '\$640 - \$660', '\$660 - \$680']


student_school_df["Spending Ranges (Per Student)"] = pd.cut(student_school_df["Per Student Budget"], spending_bins, labels = group_names)



In [21]:
dfss1 = student_school_df.groupby{["Spending Ranges (Per Student)"]}


SyntaxError: invalid syntax (<ipython-input-21-16f928a4c096>, line 1)

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.